# Chapter 6
# Function Approximation

In [ ]:
import matplotlib.pyplot as plt
import numpy as np

from matplotlib.pyplot import *
%matplotlib notebook
import matplotlib as mpl
#mpl.rcParams['savefig.dpi'] = 80
mpl.rcParams['figure.dpi'] = 80
# from IPython.display import set_matplotlib_formats
# set_matplotlib_formats('png', 'pdf')
%config InlineBackend.figure_format = 'retina'
#https://www.dataquest.io/blog/jupyter-notebook-tips-tricks-shortcuts/

import seaborn as sns
sns.set()
#sns.set_style(style= "whitegrid")
#plt.style.available
plt.style.use('fivethirtyeight')

## 
np.random.seed(123)

In many computational economic applications, one must approximate an analytically
intractable real-valued function $f$ with a computationally tractable function $\hat f$.

- In some applications, $f$ in principle can be evaluated at any point of its domain, but we wish to replace it with an approximation  $\hat f$ that is easier to work with numerically

- In other applications, $f$ is defined implicitly via a functional equation, but the equation lacks closed-form solution and we wish to compute an approximate solution $\hat f$



Two types of function approximation problems arise often in computational economic
applications. 

- In the interpolation problem, one knows the value of a function
$f$ at specified points in its domain and must choose an approximant $\hat f$ from a family
of "nice", tractable functions that matches the original function at the known evaluation
points. The interpolation problem can be *generalized* to include the value of
the function's first or higher derivatives at specified points.

1. We will first look into interpolation, a general strategy for forming a tractable approximation to a function that can be evaluated at any point of its domain


2. Then we will consider methods for solving functional equations that are based on interpolation principles

In this chapter we discuss methods for approximating functions and focus on
the two most generally practical techniques: **Chebychev polynomial and polynomial
spline approximation**. 


In addition we discuss the use of **piecewise linear functions
with finite difierence approximations for derivatives**. 


Univariate function interpolation
methods are developed first and then are generalized to multivariate function
interpolation methods. 

In the final section, we introduce the **collocation method**, a
natural generalization of interpolation methods that may be used to solve a variety
of functional equations. 

Collocation will be used extensively in Chapters 9 and 11 for
solving dynamic economic models.

## 6.1 Interpolation Principles
Interpolation involves the use of an approximating function, $\hat f$, that is easy to evaluate
in place of the function of interest, $f$. The first step in designing an interpolation
scheme is choose a family of approximating functions.

We will confine ourselves to
families of functions that can be written as a *linear combination* of a set of n linearly
independent *basis functions* $\phi_1; \phi_2; ... ; \phi_n$:


$$f(x) \approx \hat f(x)  \equiv \sum_{j=1}^n c_j{\phi}_j(x),$$

The basis coefficients $c_1, c_2,...,c_n$ can be fixed by requiring  $\hat f$ to interpolate, that is, agree with $f$, at interpolation nodes
$x_1,x_2,...x_n$ of our choosing.



For a given set of basis functions and nodes, computing the basis coefficients reduces to solving a **linear interpolation equation**

$$ \sum_{j=1}^n c_j{\phi}_j(x_i) = f(x_i),\, i = 1,2,..,n$$


The interpolation equation can also be written in the matrix form

$$ \Phi c = y $$

where

$$ \Phi_{ij} = \phi_j(x_i) \,\, \text{and} \,\, y_i = f(x_i)  $$

for $i = 1,2,..,n$  and $j = 1,2,..,n$ , and $c$ is the $n \times 1$  vector of basis coefficients to be determined

- Interpolation schemes differ only in how the basis functions $\phi_j$ and interpolation nodes $x_i$ are chosen


- In theory, an interpolation scheme is well-defined if the basis functions and interpolation nodes are chosen so that the **interpolation matrix** $\Phi$ is nonsingular

- In computational practice, however, the interpolation matrix must meet the more stringent requirement that it is not ill-conditioned

- Ideally, an interpolation scheme should also satisfy various conditions
  - It should be theoretically possible to achieve an arbitrarily accurate approximation by increasing the number of basis functions and
interpolation nodes
  - It should be possible to solve the interpolation equation quickly and accurately
  - It should be relatively inexpensive to evaluate, differentiate, integrate or otherwise work with the approximation
- In what follows, we develop interpolation schemes based on two classes of basis functions
  - orthogonal polynomials
  - piecewise polynomial splines

---

## 6.2 Polynomial Interpolation


Polynomial interpolation is motivated by the
Weierstrass theorem which asserts that any
continuous real-valued function can be
approximated over a bounded interval to an
arbitrary degree of accuracy by a polynomial



According to the Weierstrass Theorem, any continuous real-valued function f defined
on a bounded interval $[a; b]$ of the real line can be approximated to any degree of accuracy
using a polynomial. More specifically, for any $\epsilon > 0$, there exists a polynomial
$p$ such that



The Weierstrass theorem provides strong motivation for using polynomials to approximate
continuous functions. The theorem, however, is not very practical. It gives no
guidance on how to find a good polynomial approximant. It does not even state what
order polynomial is required to achieve the required level of accuracy.

#### Naïve Polynomial Interpolation

One apparently reasonable way to construct a $n^{th}$-degree polynomial approximant for a function $f$ is to form the unique $(n - 1)^{th}$-order polynomial

$$\hat f(x)  \equiv \sum_{j=1}^n c_j x^j,$$

in terms of the monomial basis functions $1, x, x^2,… , x^n$

then fix the $n + 1$ unknown basis coefficients $c_1, c_2,...,c_n$  by requiring $\hat f(x)$ to agree with $f(x)$ at the $n+ 1$ equally-spaced interpolation nodes $x_i = a + ih$, where $h = (b - a)/n$

#### Constructing the interpolation polynomial

Suppose that the interpolation polynomial is in the form

$${\displaystyle p(x)=a_{n}x^{n}+a_{n-1}x^{n-1}+\cdots +a_{2}x^{2}+a_{1}x+a_{0}.\qquad (1)} $$

The statement that p interpolates the data points means that
$${\displaystyle p(x_{i})=y_{i}\qquad {\mbox{for all }}i\in \left\{0,1,\dots ,n\right\}.}$$


If we substitute equation (1) in here, we get a system of linear equations in the coefficients ak. The system in matrix-vector form reads

$$\begin{bmatrix}
x_0^n  & x_0^{n-1} & x_0^{n-2} & \ldots & x_0 & 1 \\
x_1^n  & x_1^{n-1} & x_1^{n-2} & \ldots & x_1 & 1 \\
\vdots & \vdots    & \vdots    &        & \vdots & \vdots \\
x_n^n  & x_n^{n-1} & x_n^{n-2} & \ldots & x_n & 1
\end{bmatrix}
\begin{bmatrix} a_n \\ a_{n-1} \\ \vdots \\ a_0 \end{bmatrix}  =
\begin{bmatrix} y_0 \\ y_1 \\ \vdots \\ y_n \end{bmatrix}.$$

ref: https://en.wikipedia.org/wiki/Polynomial_interpolation

where $n$ is the polynomial degree.

Another important property of polynomials is given by the [Weierstrass Approximation Theorem](http://en.wikipedia.org/wiki/Stone%E2%80%93Weierstrass_theorem), which states given a cotinuous function $f$ defined on a interval $[a,b]$, for all $\epsilon >0$, there exits a polynomial $P(x)$ such that

$$|f(x) - P(x)|<\epsilon\ \ \ \ \  \mbox{for all }\ x\ \mbox{ in }\ [a,b].$$

This theorem guarantees the existence of such a polynomial, however it is necessary to propose a scheme to build it.

## Example

approximation $u(x)$ in the space of all linear functions:

$$\begin{equation*} V = \hbox{span}\,\{1, x\}  \end{equation*}$$

$\psi_0(x)=1$ , $\psi_1(x)=x$, and $N=1$. We seek 

$$\begin{equation*} u=c_0\psi_0(x) + c_1\psi_1(x) = c_0 + c_1x,\end{equation*}$$



In practice, however, polynomial interpolation at evenly spaced nodes often *does not* produce an accurate approximant. 

In fact, there are well-behaved functions for which polynomial approximants with evenly spaced nodes rapidly deteriorate, rather than improve, as the degree of approximation n rises.


This polynomial interpolation scheme suffers from two serious, albeit distinct problems

1. First, the interpolation matrix is a Vandermonde matrix, which becomes **increasingly ill-conditioned** as the degree of the interpolating polynomial rises

2. Second, there are functions for which the approximation **error explodes** as the degree of the interpolating polynomial rises, e.g. the Runge’s function

$$f(x) = {1 \over (1 +25x^2)} , − 1 ≤ x ≤ 1$$


3. Naïve polynomial interpolation reflects a poor choice of both the basis functions (monomials) and the interpolation nodes (equally-spaced points)


In [ ]:
from scipy.interpolate import interp1d

x = np.linspace(0, 10, num=11, endpoint=True)
y = np.cos(-x**2/9.0)
f = interp1d(x, y)
f2 = interp1d(x, y, kind='cubic')

xnew = np.linspace(0, 10, num=41, endpoint=True)

plt.figure()
plt.plot(x, y, 'o', xnew, f(xnew), '-', xnew, f2(xnew), '--')
plt.legend(['data', 'linear', 'cubic'], loc='best')
plt.title("1-D interpolation in Scipy")

#https://docs.scipy.org/doc/scipy/reference/tutorial/interpolate.html

### Runge Function

In [ ]:
x = np.linspace(-1,1, 400)
def runge(x):
    return 1/(1+25*x**2)


fig, ax = plt.subplots(1, 1,sharex=True, sharey=True);

ax.plot(x, runge(x), linewidth=2.0);
ax.set_title("Runge Function");


In [ ]:
from compecon import BasisChebyshev, BasisSpline, NLP
from compecon.quad import qnwsimp
import warnings
warnings.simplefilter('ignore')

In [ ]:
from .basis import Basis, SmolyakGrid, BasisOptions

### Table 6.1

In [ ]:
# Table 6.1
# runge = lambda x: 1 / (1 + 25 * x ** 2)
a, b = -5, 5
xx = np.linspace(a, b, 1001)
fxx = runge(xx)
def precision(fhat):
    return np.log10(np.linalg.norm(fhat - fxx, np.inf))
print('log10( norm(f(x) - fapprox(x0))) when using UNIFORM nodes')
print('\tn   monomial  chebyshev')
for n in [10, 20, 30, 40, 50]:
    # Chebyshev polynomials, uniform nodes
    Bunif = BasisChebyshev(n, a, b, f=runge, nodetype='uniform')
    # Uniform-node monomial-basis approximant
    xnodes = Bunif.nodes[0]
    c = np.polyfit(xnodes, runge(xnodes), n)
    yfit = np.polyval(c, xx)
    print('\t{:d}    {:.2f}    {:.2f}'.format(n, precision(yfit), precision(Bunif(xx))))

### Figure 6.1

In [ ]:
# # Figure 6.1
# nodes = BasisChebyshev(9, 0, 1).nodes
# plt.figure();
# plt.plot(nodes, np.zeros_like(nodes), 'r*');


#### Chebychev Polynomial Interpolation



1. Numerical analysis theory and empirical experience both suggest that polynomial approximants over a bounded interval $[a; b]$ should be constructed by interpolating the underlying function at the so-called *Chebychev nodes*:


2. The Chebychev nodes are not evenly spaced and do not include the endpoints of the approximation interval


3. They are more closely spaced near the endpoints of the approximation interval and less so near the center

For nodes over an arbitrary interval [a, b] an affine transformation can be used:

$${\displaystyle x_{k}={\frac {1}{2}}(a+b)+{\frac {1}{2}}(b-a)\cos \left({\frac {2k-1}{2n}}\pi \right),\quad k=1,\ldots ,n.}$$

In [ ]:
#Chebychev Nodes on [0,1]

a=0
b=1
n = 10
xc = (a+b)/2 + (b-a)/2*np.cos(np.pi/n*((np.arange(1,n)-.5)));     # Chebyshev nodes


In [ ]:
plt.figure()
plt.scatter(xc,xc**0, s = 125)
plt.title("Chebychev Nodes on [0,1]")
plt.xlim(-0.1, 1.1)

To illustrate the difference between Chebychev and evenly spaced node polynomial
interpolation, consider approximating the function $f(x) = exp(-x)$ on the interval
$[-1; 1]$. The approximation error associated with ten node polynomial interpolants
are illustrated in Figure 6.2. The Chebychev node polynomial interpolant exhibits
errors that oscillate fairly evenly throughout the interval of approximation, a common
feature of Chebychev node interpolants. The evenly spaced node polynomial
interpolant, on the other hand, exhibits significant instability near the endpoints
of the interval. The Chebychev node polynomial interpolant avoids such endpoint
instabilities because the nodes are more **heavily concentrated near the endpoints**.

### Figure 6.2

In [ ]:
# Figure 6.2
# 
f = lambda x: np.exp(-x)
a = -5
b = 5
xx = np.linspace(a, b, 1001)
yy = f(xx)
B = BasisChebyshev(10, -5, 5, f=f)
res_cheb = B(xx) - yy
xnodes = np.linspace(-5, 5, 10)
c = np.polyfit(xnodes, f(xnodes), 10)
res_unif = np.polyval(c, xx) - yy
plt.figure();
plt.plot(xx, res_cheb, xx, res_unif);
plt.plot(xx, np.zeros_like(xx), 'k--', linewidth=2);
plt.xlabel("x");

plt.xlim(-5,5);
plt.legend(['Chebyshev nodes', 'Uniform nodes']);
plt.title("Approximation Error for exp(−x)");

The most intuitive basis for expressing polynomials, regardless of the interpolation
nodes chosen, is the **monomial basis** consisting of the simple power functions
$1; x; x^2; x^3; : : :$, illustrated in Figure 6.3 for the interval $x \in [0; 1]$. However, the
monomial basis produces an interpolation matrix $\Phi$ that is so-called **Vandermonde maxtrix**

In linear algebra, a Vandermonde matrix, named after Alexandre-Théophile Vandermonde, is a matrix with the terms of a geometric progression in each row, i.e., an m × n matrix


$${\displaystyle \Phi={\begin{bmatrix}1&x _{1}&x _{1}^{2}&\dots &x _{1}^{n-1}\\1&x _{2}&x _{2}^{2}&\dots &x _{2}^{n-1}\\1&x _{3}&x _{3}^{2}&\dots &x _{3}^{n-1}\\\vdots &\vdots &\vdots &\ddots &\vdots \\1&x _{m}&x _{m}^{2}&\dots &x _{m}^{n-1}\end{bmatrix}},} $$


ref: https://en.wikipedia.org/wiki/Vandermonde_matrix

### Figure 6.3

In [ ]:

# Simple data to display in various forms
x = np.linspace(0, 1, 400);


# Four axes, returned as a 3-d array
fig, axarr = plt.subplots(3, 3,sharex=True, sharey=True);

axarr[0, 0].plot(x, x**0, linewidth=2.0);
#plt.ylim(0,1.1)
axarr[0, 0].set_ylim(-0.1,1.1)
axarr[0, 0].set_title(r'$1$');
axarr[0, 1].plot(x, x, linewidth=2.0);
axarr[0, 1].set_title(r'$x$');
axarr[0, 2].plot(x, x**2, linewidth=2.0);
axarr[0, 2].set_title(r'$x^2$');

axarr[1, 0].plot(x, x ** 3, linewidth=2.0);
axarr[1, 0].set_title(r'$x^3$');
axarr[1, 1].plot(x, x ** 4, linewidth=2.0);
axarr[1, 1].set_title(r'$x^4$');
axarr[1, 2].plot(x, x ** 5, linewidth=2.0);
axarr[1, 2].set_title(r'$x^5$');

axarr[2, 0].plot(x, x ** 6, linewidth=2.0);
axarr[2, 0].set_title(r'$x^6$');
axarr[2, 1].plot(x, x ** 7, linewidth=2.0);
axarr[2, 1].set_title(r'$x^7$');
axarr[2, 2].plot(x, x ** 8, linewidth=2.0);
axarr[2, 2].set_title(r'$x^8$');



# Fine-tune figure; make subplots farther from each other.
fig.subplots_adjust(hspace=0.3);

fig.suptitle("Monomial Basis Functions on [0,1]", fontsize=16)

# Tight layout often produces nice results
# but requires the title to be spaced accordingly
fig.tight_layout()
fig.subplots_adjust(top=0.88)

# Fine-tune figure; hide x ticks for top plots and y ticks for right plots
plt.setp([a.get_xticklabels() for a in axarr[0, :]], visible=False);
plt.setp([a.get_yticklabels() for a in axarr[:, 2]], visible=False);

- Interpolating at the Chebychev nodes offers many advantages
- However, merely interpolating at the Chebychev nodes does not eliminate ill-conditioning
- Ill-conditioning stems from the choice of basis functions, not the choice of interpolation nodes
- Fortunately, there is an alternative to the standard monomial basis that is ideal for interpolating at Chebychev nodes

The optimal basis for interpolating at Chebychev nodes is called the **Chebychev polynomial basis**


The Chebyshev polynomials of the first kind are defined by the recurrence relation

$${\displaystyle {\begin{aligned}T_{0}(x)&=1\\T_{1}(x)&=x\\T_{n+1}(x)&=2xT_{n}(x)-T_{n-1}(x).\end{aligned}}} $$




ref: https://en.wikipedia.org/wiki/Chebyshev_polynomials

The first few Chebyshev polynomials of the first kind are

$${\displaystyle {\begin{aligned}T_{0}(x)&=1\\T_{1}(x)&=x\\T_{2}(x)&=2x^{2}-1\\T_{3}(x)&=4x^{3}-3x\\T_{4}(x)&=8x^{4}-8x^{2}+1\\T_{5}(x)&=16x^{5}-20x^{3}+5x\\T_{6}(x)&=32x^{6}-48x^{4}+18x^{2}-1\\T_{7}(x)&=64x^{7}-112x^{5}+56x^{3}-7x\\T_{8}(x)&=128x^{8}-256x^{6}+160x^{4}-32x^{2}+1\\T_{9}(x)&=256x^{9}-576x^{7}+432x^{5}-120x^{3}+9x\\T_{10}(x)&=512x^{10}-1280x^{8}+1120x^{6}-400x^{4}+50x^{2}-1\\T_{11}(x)&=1024x^{11}-2816x^{9}+2816x^{7}-1232x^{5}+220x^{3}-11x\end{aligned}}}$$

Polynomial in Chebyshev form[edit]
An arbitrary polynomial of degree $N$ can be written in terms of the Chebyshev polynomials of the first kind. Such a polynomial $p(x)$ is of the form

$${\displaystyle p(x)=\sum _{n=0}^{N}c_{n}T_{n}(x).} $$



- Combining the Chebychev basis polynomials and Chebychev interpolation nodes yields an extremely wellconditioned interpolation equation
  - The **Chebychev interpolation matrix** is orthogonal, that is, $\Phi' \Phi$ is diagonal
  - Its condition number is $\sqrt{2}$, regardless of the degree of interpolation, which is near the absolute minimum of 1
- As a consequence, basis coefficients can be computed accurately, regardless of the degree of the approximating polynomial

When the function being approximated is very smooth, Chebychev node polynomial interpolants typically exhibit errors that oscillate fairly evenly throughout the interval of approximation

This feature is commonly referred to as the **Chebychev equi-oscillation property**

### Figure 6.4

In [ ]:
# Chebychev Polynomial Basis Functions on [0,1]


# Simple data to display in various forms
x = np.linspace(-1, 1, 400);


# Four axes, returned as a 3-d array
fig, axarr = plt.subplots(3, 3,sharex=True, sharey=True);

axarr[0, 0].plot(x, x**0, linewidth=2.0);
#plt.ylim(0,1.1)
axarr[0, 0].set_ylim(-1.9,1.9)
axarr[0, 0].set_title(r'$T_{0}(x)$');
axarr[0, 1].plot(x, 2*x, linewidth=2.0);
axarr[0, 1].set_title(r'$T_{1}(x)$');
axarr[0, 2].plot(x, 4*x**2-1, linewidth=2.0);
axarr[0, 2].set_title(r'$T_{2}(x)$');

axarr[1, 0].plot(x, 8*x ** 3-4*x, linewidth=2.0);
axarr[1, 0].set_title(r'$T_{3}(x)$');
axarr[1, 1].plot(x, 16*x ** 4-12*x**2+1, linewidth=2.0);
axarr[1, 1].set_title(r'$T_{4}(x)$');
axarr[1, 2].plot(x, 32*x ** 5-32*x**3+6*x, linewidth=2.0);
axarr[1, 2].set_title(r'$T_{5}(x)$');

axarr[2, 0].plot(x, 64*x ** 6-80*x**4+24*x**2-1, linewidth=2.0);
axarr[2, 0].set_title(r'$T_{6}(x)$');
axarr[2, 1].plot(x, 128*x ** 7-192*x**5+80*x**3-8*x, linewidth=2.0);
axarr[2, 1].set_title(r'$T_{7}(x)$');
axarr[2, 2].plot(x, 256*x ** 8-448*x**6 +240*x**4-40*x**2+1, linewidth=2.0);
axarr[2, 2].set_title(r'$T_{8}(x)$');



# Fine-tune figure; make subplots farther from each other.
fig.subplots_adjust(hspace=0.3);

fig.suptitle("Chebychev Polynomial Basis Functions on [0,1]", fontsize=18)

# Tight layout often produces nice results
# but requires the title to be spaced accordingly
fig.tight_layout()
fig.subplots_adjust(top=0.88)

# Fine-tune figure; hide x ticks for top plots and y ticks for right plots
plt.setp([a.get_xticklabels() for a in axarr[0, :]], visible=False);
plt.setp([a.get_yticklabels() for a in axarr[:, 2]], visible=False);

In [ ]:
# Table 6.2
# Errors for seletected interpolation methods
#   1: y = exp(-x)
#   2: y = 1./(1+25*x.^2).
#   3: y = sqrt(abs(x))
print('\n\n')
## Functions to be approximated
funcs = [lambda x: np.exp(-x),
         lambda x: 1 / ( 1 + 25 * x ** 2),
         lambda x: np.sqrt(np.abs(x))]
# Set degree of approximation and endpoints of approximation interval
a, b = -5, 5
x = np.linspace(a, b, 2001)  # to evaluate precision
precision = lambda y, yhat: np.max(np.abs(yhat - y))
func_names = ['exp(-x)', '1/(1+25x^2)', 'sqrt(abs(x))']
print('\t\t{:s}\t{:10s}\t{:10s}\t{:10s}'.format('n', 'Linear', 'Cubic', 'Chebyshev'))
for ii, ff in enumerate(funcs):
    fx = ff(x)
    print('\n', func_names[ii])
    for n in [10, 20, 30]:
        C = precision(BasisChebyshev(n, a, b, f=ff)(x), fx)
        S = precision(BasisSpline(n, a, b, f=ff)(x), fx)
        L = precision(BasisSpline(n, a, b, k=1, f=ff)(x), fx)
        print('\t\t{:d}\t{:.2e}\t{:.2e}\t{:.2e}'.format(n, L, S, C))

## 6.3 Piecewise Polynomial Splines

- Piecewise polynomial splines, or **simply splines for short**, are a rich, fexible class of functions that may be used instead of high degree polynomials to approximate a real-valued function over a bounded interval.

- Generally, an order $k$ spline consists of series of $k^{th}$ order polynomial segments spliced together so as to **preserve continuity of
derivatives of order $k-1$ or less.** 

The points at which the polynomial pieces are spliced together, $ v_1 < v_2 < ... < v_p$, are called the breakpoints of the spline. By convention, the first and last breakpoints are the endpoints of the interval of approximation $[a; b]$.


- Two classes of splines are often employed in practice 
  -  A first-order or linear spline is a series of **line segments** spliced together to form a continuous function
  - A third-order or cubic spline is a series of **cubic polynomials segments** spliced together to form a twice continuously differentiable function

## 6.4 Piecewise-Linear Basis Functions


- Linear splines use line segments to connect points on the graph of the function to be approximated


- They are particularly easy to construct and work with in practice, which explains their widespread popularity


- Despite their simplicity, linear splines have many virtues. For problems in which the function being approximated is not-smooth and may even exhibit discontinuities, linear splines can still provide reasonable approximations. Unfortunately, derivatives of linear splines are discontinuous, piecewise constant functions.


In [ ]:
#Interpolated method
def Interpolation( f, X, xmin, xmax, ymin=0, ymax=1, fig=None, leg=True ):
    #f(x_i) values
    Y = f( X )
    
    #X array
    Xarray = np.linspace( xmin, xmax, 1000 )
    #X area
    Xarea = np.linspace( X[0], X[-1], 1000 )
    #F array
    Yarray = f( Xarray )
    
    #Lagrange polynomial
    Ln = interp.lagrange( X, Y )
    #Interpolated array
    Parray = Ln( Xarray )
    #Interpolated array for area
    Parea = Ln( Xarea )
    
    #Plotting
    if fig==None:
        fig = plt.figure( figsize = (8,8) )
    ax = fig.add_subplot(111)
    #Function
    ax.plot( Xarray, Yarray, linewidth = 3, color = "blue", label="$f(x)$" )
    #Points
    ax.plot( X, Y, "o", color="black", label="points", zorder = 10 )
    #Interpolator
    ax.plot( Xarray, Parray, linewidth = 4, color = "red", label="$P_{%d}(x)$"%(len(X)-1) )
    #Area
    #ax.fill_between( Xarea, Parea, color="green", alpha=0.5 )
    
    #Format
    ax.set_title( "%d-point Interpolation"%(len(X)), fontsize=16 )
    ax.set_xlim( (xmin, xmax) )
    ax.set_ylim( (0, 4) )
    ax.set_xlabel( "$x$" )
    ax.set_ylabel( "$y$" )
    if leg:
        ax.legend( loc="upper left", fontsize=16 )
    ax.grid(1)
    
    return ax

#Function
def f(x):
    return 1+np.cos(x)**2+x

# Choose a region to integrate over and take only a few points in that region

X = np.array([-0.5,1.5])

#Interpolation add-on
import scipy.interpolate as interp
# Plot both the function and the interpolation 

#interpolation method
def CompositeInterpolation( f, a, b, N, n, xmin, xmax, ymin=0, ymax=1 ):
    #X array
    X = np.linspace( a, b, N )
    
    #Plotting
    fig = plt.figure( figsize = (8,8) )
    for i in range(0,N-n,n):
        Xi = X[i:i+n+1]
        ax = Interpolation( f, Xi, X[i], X[i+n], fig=fig, leg=False )
    
    #X array
    Xarray = np.linspace( xmin, xmax, 1000 )
    #F array
    Yarray = f( Xarray )
    #Function
    ax.plot( Xarray, Yarray, linewidth = 3, color = "blue", label="$f(x)$", zorder=0 )
    ax.set_title( "%d-intervals Linear Interpolation"%(N-1), fontsize=16 )
    #Format
    plt.xlim( (xmin, xmax) )
    plt.ylim( (ymin, ymax) )
    
    return None

#Quadrature with 3 points (Simpson's rule)
CompositeInterpolation( f, a=-0.9, b=1.9, N=7, n=1, xmin=-1, xmax=2, ymin=0, ymax=4 )

### Linear Spline Basis Functions

A linear spline with $n+1$ nodes $x_0, x_1,..., x_n$ on the interval $[a,b]$ may be written as  a linear combination of the $n+1$ basis functions


$$  \Phi_j(x) =
\begin{cases}
1- { |x-x_j| \over  h},  & { |x-x_j| \le h} \\
0, & \text{otherwise}
\end{cases}$$


where $h = (b-a)/n$ is the distance between the nodes evenly-spaced interpolation  




In [ ]:
ref:http://support.sas.com/documentation/cdl/en/statug/63033/HTML/default/viewer.htm#statug_introcom_a0000000525.htm

- Linear spline basis functions are often called the “hat” functions
- Each basis function is zero everywhere, except over a narrow support of width
- At most two basis functions are nonzero at any point

Computing basis coefficients for a linear spline approximation is a trivial matter

- By construction, $\phi_i(x_j)$ equals one if $i=j$, but zero otherwise, i.e. the interpolation matrix is the identity matrix $\Phi$
- Thus, the basis coefficients are simply the function values at the interpolation nodes, i.e. $c_i = f(x_i)$



Linear splines, however, possess **limitations** that make them a poor choices in most computational economic applications
- Linear splines have discontinuous first derivatives and higher order derivatives that are zero almost everywhere
- Linear splines thus do a poor job of approximating **first derivatives** and cannot approximate higher order derivatives
- In many economic applications, however, derivatives are of fundamental interest to an economist

#### Cubic Splines
- A cubic spline is a series of **cubic polynomial** segments spliced together to form a **twice continuously differentiable** function
- Cubic splines retain much of the simplicity of linear splines, but **possess continuous first and second derivatives**
- Cubic splines are therefore preferred to linear splines when a **smooth approximation** is required

In [ ]:
from scipy import interpolate
x = np.arange(0, 2*np.pi+np.pi/4, 2*np.pi/8)
y = np.sin(x)
tck = interpolate.splrep(x, y, s=0)
xnew = np.arange(0, 2*np.pi, np.pi/50)
ynew = interpolate.splev(xnew, tck, der=0)

In [ ]:
plt.figure()
plt.plot(x, y, 'b',xnew, ynew, xnew, np.sin(xnew))

plt.legend(['Linear', 'Cubic Spline', 'True'])
plt.axis([-0.05, 6.33, -1.05, 1.05])
plt.title('Cubic-spline interpolation in Scipy')
plt.show()


 - Cubic spline basis functions exhibit properties somewhat similar to linear splines
 - Each basis function is zero everywhere, except over a narrow support
 - Each basis function and its derivatives vanish at the endpoints of its support
 - At most four basis functions are nonzero at any point



In [ ]:
#https://docs.scipy.org/doc/scipy/reference/tutorial/interpolate.html
yder = interpolate.splev(xnew, tck, der=1)
plt.figure()
plt.plot(xnew, yder, xnew, np.cos(xnew),'--')
plt.legend(['Cubic Spline', 'True'])
plt.axis([-0.05, 6.33, -1.05, 1.05])
plt.title('Derivative estimation from spline in Scipy')



- Computing basis coefficients for a cubic spline approximation is also relatively easy
- By construction, at most four basis functions are nonzero at any interpolation node
- Thus, the interpolation matrix will consist mostly of zeros, with nonzero entries concentrated around the diagonal
- As such, the interpolation matrix may be stored in “sparse” format, reducing the required storage space and reducing the operations required to solve the interpolation equation
- The matrix, moreover, is naturally well-conditioned

## 6.5 Multidimensional Interpolation
The univariate interpolation methods discussed in the preceding sections may be extended
in a natural way to multivariate functions through the use of tensor products.

## 6.6 Choosing an Approximation Method

The most significant difference between spline and polynomial interpolation methods
is that spline basis functions have narrow supports, but polynomial basis functions
have supports that cover the entire interpolation interval. This can lead to big differences
in the quality of approximation when the function being approximated is
irregular. 

Discontinuities in the first or second derivatives can create problems for all
interpolation schemes. However, spline functions, due to their narrow support, can
often contain the effects of such discontinuities. Polynomial approximants, on the
other hand, allow the ill effects of discontinuities to propagate over the entire interval
of interpolation. **Thus, when a function exhibits kinks, spline interpolation may be
preferable to polynomial interpolation.**

In order to illustrate the differences between spline and polynomial interpolation,
we compare in Table 6.2 the approximation error for four different functions, all defined on $[-5; 5]$, and four different approximation schemes: linear spline interpolation,
cubic spline interpolation, evenly spaced node polynomial interpolation, and Chebychev
polynomial interpolation. The errors are measured as the maximum absolute
error using 1001 evenly spaced evaluation points on $[-5; 5]$. The approximants obtained
using splines and Chebyshev polynomials, along with the actual functions, are displayed in Figures 6.8-6.10.

### Table 6.2

In [ ]:
# Table 6.2
# Errors for seletected interpolation methods
#   1: y = exp(-x)
#   2: y = 1./(1+25*x.^2).
#   3: y = sqrt(abs(x))
print('\n\n')
## Functions to be approximated
funcs = [lambda x: np.exp(-x),
         lambda x: 1 / ( 1 + 25 * x ** 2),
         lambda x: np.sqrt(np.abs(x))]
# Set degree of approximation and endpoints of approximation interval
a, b = -5, 5
x = np.linspace(a, b, 2001)  # to evaluate precision
precision = lambda y, yhat: np.max(np.abs(yhat - y))
func_names = ['exp(-x)', '1/(1+25x^2)', 'sqrt(abs(x))']
print('\t\t{:s}\t{:10s}\t{:10s}\t{:10s}'.format('n', 'Linear', 'Cubic', 'Chebyshev'))
for ii, ff in enumerate(funcs):
    fx = ff(x)
    print('\n', func_names[ii])
    for n in [10, 20, 30]:
        C = precision(BasisChebyshev(n, a, b, f=ff)(x), fx)
        S = precision(BasisSpline(n, a, b, f=ff)(x), fx)
        L = precision(BasisSpline(n, a, b, k=1, f=ff)(x), fx)
        print('\t\t{:d}\t{:.2e}\t{:.2e}\t{:.2e}'.format(n, L, S, C))

 - Chebychev polynomial interpolation tends to outperform spline interpolation when the function being approximated is very **smooth**
 - However, if the function possesses discontinuities in the first or second derivative, spline functions sometimes perform as well or better
 - Also, if the dimension of the problem is large, spline interpolation enjoys an advantage because of its **sparse interpolation matrix**

## 6.7 An Approximation Toolkit


Implementing routines for multivariate function approximation involves a number of
bookkeeping details that are tedious at best. In this section we describe a set of
numerical tools that take much of the pain out of this process. This toolbox contains
several high-level functions that use a structured variable to store the essential information
that defines the function space from which approximants are drawn. The `CompEcon-python`
toolbox also contains a set of middle-level routines that define the basis functions for
Chebychev polynomials and for splines and a set of low-level utilities to handle basic
computations, including tensor product manipulations.




https://github.com/randall-romero/CompEcon-python/tree/master/notebooks/app




In [ ]:
from compecon import BasisChebyshev,nodeunif
from mpl_toolkits.mplot3d import Axes3D
from matplotlib import cm

## Univariate approximation

Approximating the function $f(x) = e^{-2x}$. Its derivative is $f'(x) = -2e^{-2x}$

In [ ]:
f1 = lambda x: np.exp(-2 * x)
d1 = lambda x: -2 * np.exp(-2 * x)

### Fit approximant

The CompEcon toolbox defines the ```BasisChebyshev``` class for Chebyshev interpolation. 

Its positional arguments are 
 - `n` (number of nodes), 
 - `a` (lower bound) and `b` (upper bound). 
 - The optional keyword argument `f` indicates a function (the lambda `f1` in our example) to be approximated.

In [ ]:
n, a, b = 10, -1, 1
f1fit = BasisChebyshev(n, a, b, f=f1)

Here, `f1fit` is an instance of the `BasisChebyshev` class. 

Once a function is specified (as with the keyword argument `f` above), it can be evaluated at a given vector `x` by *calling* `f1fit`  as any other function


```python
f1fit()   # without arguments, it evaluates the function at the basis nodes
f1fit(x)  # returns a vector containing the interpolation of each element of x
```

When a function is specified with the option `f`, the `BasisChebyshev` object computes the interpolation coefficients $c = \Phi(x)^{-1}f(x)$, where $x$ represent the nodes of bhe basis.  

Alternatively, if the values `fx` of the function at the nodes are available (instead of the function itself, as is usually the case), then the basis is created by:

```python
BasisChebyshev(n, a, b, y=fx)
```

In [ ]:
f1fit()

In [ ]:
x = np.linspace(-1,1,10)
f1fit(x)

In [ ]:
plt.figure()
plt.plot(x,f1(x),label = r'Function: $e^{-2x}$')
plt.plot(np.linspace(-1,1,10) ,f1fit(),'-*', label = "Approximation")
plt.legend()

#### Chebychev polynomial and spline approximantion of various functions

Demonstrates Chebychev polynomial, cubic spline, and linear spline approximation for the following functions
\begin{align}
    y &= 1 + x + 2x^2 - 3x^3 \\
    y &= \exp(-x) \\
    y &= \frac{1}{1+25x^2} \\
    y &= \sqrt{|x|} 
\end{align}

ref: https://cybera.syzygy.ca/jupyter/user/georgebctt/notebooks/econ/CompEcon-python/notebooks/app/05%20Chebychev%20polynomial%20and%20spline%20approximantion%20of%20various%20functions.ipynb

In [ ]:
from compecon import BasisChebyshev, BasisSpline, nodeunif

In [ ]:
funcs = [lambda x: 1 + x + 2 * x ** 2 - 3 * x ** 3,
         lambda x: np.exp(-x),
         lambda x: 1 / ( 1 + 25 * x ** 2),
         lambda x: np.sqrt(np.abs(x))]

fst = ['$y = 1 + x + 2x^2 - 3x^3$', '$y = \exp(-x)$', 
       '$y = 1/(1+25x^2)$', '$y = \sqrt{|x|}$']

In [ ]:
x = nodeunif(2001, a, b)

def subfig(k, x, y, xlim, ylim, title):
    plt.subplot(2, 2, k)
    plt.plot(x, y)
    plt.xlim(xlim)
    plt.ylim(ylim)
    plt.title(title)

In [ ]:
n = 7   # degree of approximation
a = -1  # left endpoint
b = 1   # right endpoint

In [ ]:
def plot_appro( n, a, b,x, ifunc, ff):   
    """Plot approximation for Figure 6.8 -6.10
    ifunc: number of function
    ff: function
    n = 7   # degree of approximation
    a = -1  # left endpoint
    b = 1   # right endpoint 
    """
    
    # Construct interpolants
    C = BasisChebyshev(n, a, b, f=ff)
    S = BasisSpline(n, a, b, f=ff)
    L = BasisSpline(n, a, b, k=1, f=ff)

    # Compute actual and fitted values on grid
    y = ff(x)  # actual
    yc = C(x)  # Chebychev approximant
    ys = S(x)  # cubic spline approximant
    yl = L(x)  # linear spline approximant

    
    # Plot function approximations
    plt.figure(figsize=[12,8])
    ymin = np.floor(y.min())
    ymax = np.ceil(y.max())
    xlim = [a, b]
    ylim = [-0.2, 1.2] if ifunc==2 else [ymin, ymax]

    subfig(1, x, y, xlim, ylim, fst[ifunc])
    subfig(2, x, yc, xlim, ylim, 'Chebyshev')
    subfig(3, x, ys, xlim, ylim, 'Cubic Spline')
    subfig(4, x, yl, xlim, ylim, 'Linear Spline')

    # Plot function approximation error
    plt.figure()
    plt.plot(x, np.c_[yc - y, ys - y], linewidth=3)
    plt.xlabel('x')
    plt.ylabel('Approximation Error')
    plt.legend(['Chebychev Polynomial','Cubic Spline'])
    # pltlegend('boxoff')

In [ ]:
plot_appro(n, a, b,x, 0, funcs[0],)

#### Figure 6.8

Chebyshev better 

In [ ]:
plot_appro(n, a, b,x,1, funcs[1])

#### Figure 6.9

In [ ]:
plot_appro(n, a, b,x,2, funcs[2])

#### Figure 6.10

Linear Spline better

In [ ]:
plot_appro(n, a, b,x,3, funcs[3])

#### Example page 139-140

In [ ]:
# Example page 139-140
alpha = 2
f = lambda x: np.exp(-alpha * x)
F = BasisChebyshev(10, -1, 1, f=f)
x = np.linspace(-1, 1, 1001)
plt.figure()
plt.plot(x, F(x) - f(x))
plt.title('Figure 6.11  Approximation Error')
#plt.show()


## 6.8 The Collocation Method
In this section we introduce the *collocation method*, a straightforward generalization of the
function approximation methods covered earlier in this chapter that can be used to solve
a wide variety of functional equations, including the functional equations that arise with
dynamic economic models in discrete and continuous time.






## 6.9 Boundary Value Problems
In the boundary value problem, or BVP for short, one seeks a solution function $x(t) : [0; T] \rightarrow
R^d$ that satisfies the differential equation

In [ ]:

#### Example p147

In [ ]:
# # Example p147

# from compecon import BasisChebyshev, BasisSpline, NLP
# warnings.simplefilter('ignore')
# r, k, eta, s0 = 0.1, 0.5, 5 ,1


# T, n = 1, 15
# tnodes = BasisChebyshev(n - 1, 0, T).nodes
# F = BasisChebyshev(n, 0, T, y=np.ones((2, n)))


# def resid(c, tnodes, T, n, F, r, k, eta, s0):
#     F.c = np.reshape(c[:], (2, n))
#     (p, s), d = F(tnodes, [[0, 1]])
#     d[0] -= (r * p + k)
#     d[1] += p ** -eta
#     (p_0, p_T), (s_0, s_T) = F([0, T])
#     return np.r_[d.flatten(), s_0 - s0, s_T]


# storage = NLP(resid, F.c.flatten(), tnodes, T, n, F, r, k, eta, s0)
# c = storage.broyden(print=True)
# F.c = np.reshape(c, (2, n))

# nplot = 501
# t = np.linspace(0, T, nplot)
# (p, s), (dp, ds) = F(t, [[0, 1]])
# res_p = dp - r * p - k
# res_s = ds + p ** -eta
# plt.figure()
# plt.subplot(2, 1, 1)
# plt.plot(t, res_p)
# plt.title('Residuals')
# plt.ylabel('d(price) residual')

# plt.subplot(2, 1, 2)
# plt.plot(t, res_s)
# plt.xlabel('time')
# plt.ylabel('d(storage) residual')


# plt.figure()
# plt.subplot(2, 1, 1)
# plt.plot(t, p)
# plt.title('Solution')
# plt.ylabel('Price')

# plt.subplot(2, 1, 2)
# plt.plot(t, s)
# plt.xlabel('time')
# plt.ylabel('Stock')


# plt.show()

## 6.10 Scipy Interpolation

Given a set of N points $(x_i, y_i)$ with $i = 1, 2, …N$, we use a function $\hat{f}(x)$ which provides interpolation of the data $(x_i, y_i)$ for all $x$ in an interval $[a,b]$.

The function `y0 = scipy.interpolate.interp1d(x,y,kind=’nearest’)` does this interpolation based on splines of varying order. Note that the function `interp1d` returns a function `y0` which will then interpolate the x-y data for any given $x$ when called as $y0(x)$.
The code below demonstrates this, and shows the different interpolation kinds.

In [ ]:
import scipy.interpolate

def create_data(n):
    """Given an integer n, returns n data points
    x and values y as a numpy.array.
    True function: f(x) = -x^2
    y is the observations with normal noise
    """
    xmax = 5.
    x = np.linspace(0, xmax, n)
    y = - x**2
    #make x-data somewhat irregular
    y += 3.5 * np.random.normal(size=len(x))
    return x, y

#main program
n = 10
x, y = create_data(n)

#use finer and regular mesh for plot
xfine = np.linspace(0.1, 4.9, n * 100)

# using n=10 breakpoints to do the intepolation
#interpolate with piecewise constant function (p=0): step function
y0 = scipy.interpolate.interp1d(x, y, kind='nearest')
#interpolate with piecewise linear func (p=1): piecewise straight line
y1 = scipy.interpolate.interp1d(x, y, kind='linear')
#interpolate with piecewise quadratic func (p=2): piecewise curve 
y2 = scipy.interpolate.interp1d(x, y, kind='quadratic')
#interpolate with piecewise cubic func (p=3): piecewise curve 
y3 = scipy.interpolate.interp1d(x, y, kind='cubic')
# true function
yt = - xfine**2



In [ ]:
sns.set_style(style= "whitegrid")
#plt.style.available
#plt.style.use('fivethirtyeight')

plt.figure()
plt.scatter(x, y,s = 150,label='data point')
plt.plot(xfine, y0(xfine), label='nearest')
plt.plot(xfine, y1(xfine), label='linear')
#plt.plot(xfine, y2(xfine), label='quadratic')
#plt.plot(xfine, y3(xfine), label='cubic')
plt.plot(xfine, yt, label='True function')


plt.legend()
plt.xlabel('x')

In [ ]:
plt.figure()
plt.scatter(x, y, s = 150,label='data point')
plt.plot(xfine, y0(xfine), label='nearest')
#plt.plot(xfine, y1(xfine), label='linear')
#plt.plot(xfine, y2(xfine), label='quadratic')
plt.plot(xfine, y3(xfine), label='cubic')
plt.plot(xfine, yt, label='True function')


plt.legend()
plt.xlabel('x')